# NB3: Notebook created for extarcting all the necessary features from the IUROPA DB
**Output:** CREATE  a features dataframe that can be easily integrated with the main dyadic unit of measurement or the main dyadic unit of analysis.

## Environment and Visual Settings


In [3]:
import os 
import sys
import zipfile
import numpy as np
import pandas as pd
import pandasql as ps
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

# ADD project paths for for easier file access across whole project
#paths_script_path = "C:\\cloudgit\\uio24emc\\"
## CREATE parent dir path
parent_dir = os.path.dirname(os.getcwd(),) +"\\"
## ADD a dir to the list of directories when Python importing modules.
sys.path.append(parent_dir)
## IMPORTS a paths file that lives in your ./root directory 
import paths 

#$ EXAMPLE of how to access paths globally
print(paths.tex_fig_dir)

# Import the functions from custom scripts
from  costum_py import check_match, create_unique_id

C:\Users\mauricmm\UiO Dropbox\Mauricio Mandujano Manriquez\Apps\Overleaf\DRAFT_uio24emc_referral_failure_mmmanriquez\figures


## Data Collection

### IUROPA DS Variable Selector

HERE Goes the description of the per TABLE from the IUROPA DB


In [45]:
##SAVE IN R Lessons for a Python user 


array(['PROC_AA', 'PROC_INTP', 'PROC_AD', 'PROC_SC', 'PROC_AFA',
       'PROC_AP', 'PROC_REV', 'PROC_AFFO', 'PROC_RPR', 'PROC_R',
       'PROC_AC', 'PROC_P', 'not available', 'PROC_OP', 'PROC_INTV',
       'PROC_RMI', 'PROC_RX', 'PROC_MI', 'PROC_DEP'], dtype=object)

In [46]:
## CREATE Path tp zip directroy I like using functional strings than os. for joining paths
iuripa_zip_path = os.path.join(paths.raw_dir, "IUROPA-CJEU-Database-Platform.zip"  )
iuropa_path = f"{paths.raw_dir}IUROPA-CJEU-Database-Platform"
iuropa_db_cvs_paths = (os.listdir(iuropa_path))

# Start reading at FOR: Let {csv} be every item in {iuropa_db_cvs_paths} return {indexing_csv} if {this text} is in {indexing_csv}
matching_items = [csv for csv in iuropa_db_cvs_paths   if "cases" in csv]
print(matching_items)

## READ ALL REQUIRED IUROPA DB CSV
### BELOW features are easy to broadcast to unit of analysis 
iu_nat_courts_ds = pd.read_csv(f"{paths.raw_dir}IUROPA-CJEU-Database-national-courts.csv")
iu_assignments_ds = pd.read_csv( f"{iuropa_path}\\IUROPA-CJEU-Database-Platform-assignments.csv")
iu_judgements_ds = pd.read_csv( f"{iuropa_path}\\IUROPA-CJEU-Database-Platform-judgments.csv")

###BELOW features you need to really figure out what is what (:S)
iu_referrals_ds = pd.read_csv( f"{iuropa_path}\\IUROPA-CJEU-Database-Platform-referrals.csv")
iu_proceedings_ds = pd.read_csv( f"{iuropa_path}\\IUROPA-CJEU-Database-Platform-proceedings.csv")
iu_procedures_ds = pd.read_csv( f"{iuropa_path}\\IUROPA-CJEU-Database-Platform-procedures.csv")
iu_p_rulings_ds = pd.read_csv( f"{iuropa_path}\\IUROPA-CJEU-Database-Platform-preliminary-rulings.csv")
iu_p_cases_ds = pd.read_csv( f"{iuropa_path}\\IUROPA-CJEU-Database-Platform-cases.csv")
iu_judges_ds = pd.read_csv( f"{iuropa_path}\\IUROPA-CJEU-Database-Platform-judges.csv")

# SUBSET for PRELIMINARY REFERENCES ONLY
iu_procedures_ds = iu_procedures_ds.query("iuropa_procedure_id == 'PROC_RPR'")

############################################
# <> FUNCTION creates lead case for cases table.
############################################
# CREATE ALL Cases for bundling procedures broadcasting case-level outcomes
## Create dictionary to query and return id paris
id_value_query = dict(zip(iu_p_cases_ds['joined_to'], iu_p_cases_ds['cjeu_case_id']))
id_value_return = dict(zip(iu_p_cases_ds['cjeu_case_id'], iu_p_cases_ds['iuropa_case_id']))

## RUN Loop to replace values and have true lead case.
iu_p_cases_ds['iuropa_lead_case'] = 0 

for index, row in iu_p_cases_ds.iterrows():
    if row.is_joined == 0 :
        iu_p_cases_ds.at[index, 'iuropa_lead_case'] = str(row['iuropa_case_id'])
    else:
        # Use Dictionary mappings for 
        valin =  id_value_query[row.joined_to] 
        valout = id_value_return[valin]
         # print(f"Repalced {valin} for {valout}")
        iu_p_cases_ds.at[index , 'iuropa_lead_case'] = valout

############################################
# <>END of FUNCTION 
############################################

#######################
# DONE reading dataframes
#######################
# INFO SETS(iuropa_case_id from iu_p_rulings_ds == iuropa_case_id from iu_referrals_dss  iuropa_case_id)

['IUROPA-CJEU-Database-Platform-cases.csv']


### QUERY Dataframes 

FROM TABLE 

In [31]:
iu_judges_ds

# iu_p_rulings_ds.unique()
iu_p_rulings_ds.columns

Index(['key_id', 'iuropa_court_id', 'court', 'iuropa_case_id', 'cjeu_case_id',
       'case_year', 'case_number', 'iuropa_proceeding_id',
       'cjeu_proceeding_id', 'iuropa_proceeding_suffix_id',
       'proceeding_suffix', 'proceeding_name', 'count_joined_proceedings',
       'list_joined_proceedings', 'count_decisions', 'list_decisions',
       'uses_urgent_procedure', 'count_referring_member_states',
       'list_referring_member_states', 'count_referring_national_courts',
       'list_referring_national_courts', 'list_documents_interpreted',
       'list_documents_declared_valid', 'list_documents_declared_void'],
      dtype='object')

In [57]:
######################################
# TABLE1 KEY IDs at Unit of Anlalyis  refs >-to-one case
######################################
# FROM REFERRALS  SELECT cjeu_case_id== UoA
iu_referrals_ds[[ 'iuropa_case_id','iuropa_national_court_id']]
# FROM CASES SELECT cjeu_case_id== UoA== iuropa_case_id
iu_p_cases_ds[['iuropa_lead_case','iuropa_case_id']]
# FROM iu_referrals_ds LEFT JOIN ON 'iuropa_case_id'
iu_uio24emc= f"{paths.raw_dir}iu_uio24emc\\"
######################################
# TABLE2 UoA Features 
######################################
# FROM REFERRALS  SELECT cjeu_case_id== UoA== iuropa_case_id
iu_referrals_ds[['cjeu_case_id', 'iuropa_case_id',  'iuropa_decision_id', 'uses_urgent_procedure', 'iuropa_national_court_id','iuropa_court_id', 'court' ]]
# FROM CASES SELECT cjeu_case_id== UoA== iuropa_case_id
iu_p_cases_ds[['iuropa_lead_case','iuropa_case_id', 'cjeu_case_id',  'count_proceedings', 'case_name', 'is_joined', 'joined_to', 'is_pending',  'is_removed', 'has_opinion' , 'has_judgment', 'list_proceedings']]
# FROM iu_referrals_ds LEFT JOIN ON 'iuropa_case_id'

#FROM NATIONAL courts SELECT
## RENAME to avoid confusion
iu_nat_courts_ds.rename(columns= {'iuropa_member_state_id' : 'referring_member_state_id' }, inplace=True )
## CREATE  ['lower court', 'higher court', 'not applicable', 'mid-level court']
court_usedto_rev_encoding = {'lower court': 1, 'mid-level court': 2, 'higher court': 3, 'not applicable': 0}
iu_nat_courts_ds['court_usedto_rev'] = iu_nat_courts_ds['court_level'].map(court_usedto_rev_encoding)
iu_nat_courts_ds[['iuropa_national_court_id', 'national_court',  'referring_member_state_id', 'member_state', 'english_translation', 'in_hierarchy', 'court_level', 'court_collegiality', 'court_competence' ]]
# FROM key_ids_emc LEFT JOIN ON 'iuropa_national_court_id'


######################################
# TABLE3  Case LVL features broadcasted intoUoA 
######################################
#FROM NATIONAL courts SELECT
## RENAME variable for proper merging
iu_p_rulings_ds.rename(columns= {'iuropa_case_id' : 'iuropa_lead_case' }, inplace=True )
iu_p_rulings_ds.rename(columns= {'proceeding_name' : 'lead_proceeding_name' }, inplace=True )
iu_p_rulings_ds.rename(columns= {'iuropa_proceeding_id' : 'lead_iuropa_proceeding_id' }, inplace=True )

iu_p_rulings_ds[['iuropa_lead_case',  'lead_proceeding_name',  'case_year', 'lead_iuropa_proceeding_id' ,'count_joined_proceedings', 'list_joined_proceedings', 
                 'count_decisions', 'uses_urgent_procedure' , 'count_referring_member_states','count_referring_member_states','list_referring_member_states',  'count_referring_national_courts', ]]
#FROM key_ids_emc EFT JOIN ON 'iuropa_lead_case'



# [ ] Proceedings TABLE case lvl...
# [ ] Procedures  TABLE at what lvl...
# [ ] TODO SQL CREATE TABLE 2 For UoA features

######################################
# TABLE4 Case LVL features not necessary to be broadcasted.
######################################
iu_p_rulings_ds[['iuropa_lead_case', 'list_documents_interpreted','list_documents_declared_valid', 'list_documents_declared_void', 'list_decisions']]


######################################
# TABLE5  Judges features by iuropa_lead_case
######################################
# FROM ASSINGMENTS  SELECT  features at LEAD CASE Level
## CREATE iuropa_lead_case for easier merger
iu_assignments_ds['iuropa_lead_case'] = iu_assignments_ds['iuropa_case_id']
iu_assignments_ds[['iuropa_lead_case','iuropa_judge_id', 'is_judge_rapporteur']] 
# FROM iu_p_cases_ds LEFT JOIN  ON 'iuropa_lead_case' 

# FROM  JUDGES SELECT 
## CREATE for avoid confounding 
iu_judges_ds.rename(columns= {'iuropa_member_state_id' : 'appointing_member_state_id' }, inplace=True )
iu_judges_ds[['iuropa_judge_id' , 'full_name', 'appointing_member_state_id', 'is_female', 'was_judge', 'was_academic' ]]
# FROM iu_assignments_ds LEFT JOIN  ON iuropa_judge_id
# NOTE Using this table you can create a case level feature tobe joined on 'iuropa_lead_case'  SELECT  'full_name', 'appointing_member_state_id' 'is_female', 'was_judge', 'was_academic'  WHEN is_judge_rapporteur==1 
# This would eneable you to do more analysis IF you wanted to. AT least  'appointing_member_state_id' matches 'referring court' could be a cue of context. 



######################################
# TABLE6 For scraping applications
######################################
# FROM  PROCEDURES SELECT [['iuropa_case_id' , 'celex']]
scraping_celexes = iu_procedures_ds[['iuropa_case_id' , 'celex']]
apps_scraping = pd.merge(key_ids_emc, scraping_celexes, how='left' ,on='iuropa_case_id')
apps_scraping.rename(columns={'celex':'judgment_celex'} ,inplace=True) 
apps_scraping['apps_celex']= apps_scraping['judgment_celex'].str.replace('CJ', 'CN')

##CREATE splitting index
split_index = apps_scraping[apps_scraping['apps_celex']=='62008CN0001' ].index[0]
apps_scraping['eurlex_available']= (apps_scraping.index >=split_index).astype(int)


apps_scraping.to_csv(f"{iu_uio24emc}apps_scraping.csv" , index=False)

######################################
# TABLE7
######################################


print("WERE HERE ")
# REMEMBER: pseudo keyid is iuropa_case_id for now
# MERGE using iuropa_case_id ids
# [ ] CREATE unique referralID using  IUROPA CASE  iuropa_case_id AND iuropa_national_court_id

# CHECK exact match two columns: check_match(iu_referrals_ds, 'cjeu_case_id', 'cjeu_proceeding_id') [['cjeu_case_id', 'cjeu_proceeding_id']]
# USE .isin() when you wnat multiple values from one column.
# USE df[df['A'] > 1] when 
# USE duplicate_count = df.duplicated().sum() to quickly count duplicates.
# USE for creating row based dummy flags 
# split_index = apps_scraping[apps_scraping['apps_celex']=='62008CN0001' ].index[0]


WERE HERE 


In [56]:
apps_scraping

,iuropa_lead_case,uoa_dyad_id,iuropa_national_court_id,iuropa_case_id,judgment_celex,apps_celex
0,CASE_C_1961_0013,REF_1961_0013NE,NAT_COURT_NE_GH_GRA,CASE_C_1961_0013,61961CJ0013,61961CN0013
1,CASE_C_1962_0026,REF_1962_0026NE,NAT_COURT_NE_TC,CASE_C_1962_0026,61962CJ0026,61962CN0026
2,CASE_C_1962_0028,REF_1962_0028NE,NAT_COURT_NE_TC,CASE_C_1962_0028,61962CJ0028,61962CN0028
3,CASE_C_1963_0073,REF_1963_0073NE,NAT_COURT_NE_CBB,CASE_C_1963_0073,61963CJ0073,61963CN0073
4,CASE_C_1963_0075,REF_1963_0075NE,NAT_COURT_NE_CRB,CASE_C_1963_0075,61963CJ0075,61963CN0075
...,...,...,...,...,...,...
8759,CASE_C_2022_0659,REF_2022_0659CZ,NAT_COURT_CZ_NSS,CASE_C_2022_0659,62022CJ0659,62022CN0659
8760,CASE_C_2022_0682,REF_2022_0682HR,NAT_COURT_HR_US_ZAG,CASE_C_2022_0682,62022CJ0682,62022CN0682
8761,CASE_C_2022_0700,REF_2022_0700CZ,NAT_COURT_CZ_NS,CASE_C_2022_0700,62022CJ0700,62022CN0700
8762,CASE_C_2023_0001,REF_2023_0001BE,NAT_COURT_BE_TPIF_BRU,CASE_C_2023_0001,62023CJ0001,62023CN0001


In [ ]:
iu_referrals_ds.columns
iu_proceedings_ds.columns
iu_procedures_ds.columns
iu_p_rulings_ds.columns
compare_list = [iu_referrals_ds, iu_proceedings_ds,iu_p_rulings_ds, iu_procedures_ds ]
print(iu_referrals_ds[['iuropa_proceeding_id']].shape)
print(iu_referrals_ds[['iuropa_proceeding_id']].nunique())

#This is okay but now way to knwo which ones are referrals
cdings_pro_ids = set(iu_proceedings_ds['iuropa_proceeding_id'].unique())
refs_pro_ids = set(iu_referrals_ds['iuropa_proceeding_id'].unique())
cdurs_pro_ids = set(iu_procedures_ds['iuropa_proceeding_id'].unique())


cdurs_pro_ids

len(cdurs_pro_ids)
cdurs_pro_ids.columns


(8762, 1)
iuropa_proceeding_id    8608
dtype: int64


AttributeError: 'set' object has no attribute 'columns'

In [14]:
# FROM CASES get all c-procedings before the court 
#iu_p_cases_ds[['iuropa_case_id', 'cjeu_case_id', 'case_name', 'is_joined', 'joined_to', 'is_pending',  'is_removed', 'has_opinion' , 'has_judgment' ]]


c:\gitprojects\uio24emc\raw\uio24emc


In [37]:
#######################
# CREATE QUERIES for Tables 
######################
iu_uio24emc= f"{paths.raw_dir}iu_uio24emc\\"
###########################################
# TABLE1 KEY IDs at Unit of Anlalyis  refs >-to-one case
###########################################
# query_t1 ="""
# SELECT
#     iu_referrals_ds.iuropa_case_id,
#     iu_referrals_ds.iuropa_national_court_id,
#     iu_p_cases_ds.iuropa_lead_case
# FROM iu_referrals_ds
# LEFT JOIN iu_p_cases_ds ON iu_referrals_ds.iuropa_case_id = iu_p_cases_ds.iuropa_case_id
# """ 

# key_ids_emc = ps.sqldf(query_t1, locals())
# # SAFELY drop duplicates 
#key_ids_emc = key_ids_emc.drop_duplicates(keep='first')

##  CREATE UNIQUE ID
key_ids_emc['uoa_dyad_id'] = key_ids_emc.apply(create_unique_id, axis=1)
key_ids_emc= key_ids_emc[['iuropa_lead_case','uoa_dyad_id', 'iuropa_national_court_id','iuropa_case_id']]
##################
#SAVE TABLE 1
##################
key_ids_emc.to_csv(f"{iu_uio24emc}key_ids_emc.csv" , index=False)

###########################################
# TABLE2 UoA Features 
###########################################
query_t2 = """
SELECT  
FROM df1
LEFT JOIN df2
ON df1.id = df2.id

"""

###########################################
# TABLE2 
###########################################
query_t3 = """
SELECT  
FROM df1
LEFT JOIN df2
ON df1.id = df2.id

"""



######################################
# TABLE6 For scraping applications
######################################
# FROM  PROCEDURES SELECT [['iuropa_case_id' , 'celex']]
scraping_celexes = iu_procedures_ds[['iuropa_case_id' , 'celex']]
apps_scraping = pd.merge(key_ids_emc, scraping_celexes, how='left' ,on='iuropa_case_id')
##################
#SAVE TABLE 6
# ##################
apps_scraping.to_csv(f"{iu_uio24emc}apps_scraping.csv" , index=False)




In [ ]:

## FINAL QUERY PERFORM SQL  Query than returns only desired features and data
query = """
SELECT  
FROM df1
LEFT JOIN df2
ON df1.id = df2.id

"""


In [ ]:
matching_rows = iu_procedures_ds[iu_procedures_ds['iuropa_procedure_id'].str.contains( "not" , case=False, na=False)]
mask = iu_procedures_ds['iuropa_procedure_id'].isin( ['not available', 'PROC_P'])

## TRASH BIN


In [ ]:
# Nevermind.
# what is the most efficent way to achieve the following::
# assume I am in datframe 'iu_p_cases_ds'
# I wnat to create a new column name "lead_case_id" 
# When  column 'cjeu_case_id' != 'not applicable' find this value in column ' '

In [34]:
# # Function to read CSV with multiple encoding attempts

# with zipfile.ZipFile(iuripa_zip_path, 'r',) as z:
#     # List all files in the zip archive
#     file_list = z.namelist()
    
#     # Filter out only CSV files But exludes for the MAC OS csvs.
#     csv_files = [f for f in file_list if f.endswith('.csv') and not f.startswith('__MACOSX')]
    
#     print(csv_files)
#     # Iterate over CSV files and read them into DataFrames
#     for csv_file in csv_files:
#         with z.open(csv_file) as f:
#             # Read the CSV file into a DataFrame
#             df = pd.read_csv(f)

# # Print the DataFrames 
# # for file_name, df in dataframes.items():
# #     print(f"DataFrame for {file_name}:")

In [ ]:
# iu_p_cases_ds.columns
# iu_referrals_ds.columns
# iu_proceedings_ds.columns
# iu_procedures_ds.columns
# iu_p_rulings_ds.columns
# compare_list = [iu_referrals_ds, iu_proceedings_ds,iu_p_rulings_ds, iu_procedures_ds ]
# print(iu_referrals_ds[['iuropa_proceeding_id']].shape)
# print(iu_referrals_ds[['iuropa_proceeding_id']].nunique())


# # {cdings_pro_ids} is okay but NO way to know which ones are PRELIMINARY RULINGS 
# cdings_pro_ids = set(iu_proceedings_ds['iuropa_proceeding_id'].unique())

# refs_pro_ids = set(iu_referrals_ds['iuropa_proceeding_id'].unique())
# prs_cdurs_pro_ids = set(iu_procedures_ds['iuropa_proceeding_id'].unique())
# prs_pro_ids = set(iu_p_rulings_ds['iuropa_proceeding_id'].unique())


# #CHECK IF  iuropa_procedure_id is UNIQUE for dyad but I think not
# print( "PROCEDURES Len:", len(prs_cdurs_pro_ids))


# iu_procedures_ds["iuropa_proceeding_id"].nunique() 
# iu_procedures_ds["iuropa_procedure_id"].nunique()

# dup_df =  iu_referrals_ds[['iuropa_proceeding_id','cjeu_proceeding_id','iuropa_national_court_id', 'cjeu_case_id',]]

# check = dup_df[dup_df.duplicated(subset=['iuropa_proceeding_id'], keep=False)]


# CHECK if  iuropa_procedure_id is your unique id
#iu_procedures_ds[['cjeu_proceeding_id','iuropa_proceeding_id']]
#dup_df[dup_df.duplicated(subset=['iuropa_case_id', 'cjeu_case_id'], keep=False)]
# COUNT row_uniquevalues: iu_procedures_ds["iuropa_proceeding_id"].value_counts()
# COUNT col_uniquevalues: iu_referrals_ds[['iuropa_proceeding_id', 'iuropa_case_id', 'cjeu_case_id',]].nunique()
# CHECK  for duplicates based on one column and set {keep=True} to keep  duplicated records: dup_df[dup_df.duplicated(subset=['iuropa_proceeding_id'], keep=True)]
# CHECK for duplicates based on a set of columns as if they are an unique ID and  set {keep=True} to keep  duplicated records : dup_df[dup_df.duplicated(subset=['colone','col2'], keep=True)]